In [0]:
###############################################################################
# SETUP - EXECUTE THIS FIRST                                                  #
###############################################################################
# 1. Go into "Runtime" -> "Change runtime type" and Select "GPU" for hardward accelerator
# 2. Click the "Connect" button, at the right to start the instance.
# This will get the dataset into this instance
!wget https://github.com/nknewlands/DeepLearningShortCourse/raw/master/Day1/EuroSat_Classification/EuroSatRGB_very_small.zip
!unzip EuroSatRGB_very_small.zip

--2019-08-12 14:13:12--  https://github.com/nknewlands/DeepLearningShortCourse/raw/master/Day1/EuroSat_Classification/EuroSatRGB_very_small.zip
Resolving github.com (github.com)... 192.30.253.113
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/nknewlands/DeepLearningShortCourse/master/Day1/EuroSat_Classification/EuroSatRGB_very_small.zip [following]
--2019-08-12 14:13:13--  https://raw.githubusercontent.com/nknewlands/DeepLearningShortCourse/master/Day1/EuroSat_Classification/EuroSatRGB_very_small.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7013674 (6.7M) [application/zip]
Saving to: ‘EuroSatRGB_very_small.zip’

EuroSatRGB_very_sma 100%[==

In [0]:
#!/usr/bin/python
# ==============================================================================
# This demonstrate how to use transfer learning using a ResNet50 network.
# Original data from https://arxiv.org/abs/1709.00029
# Etienne Lord - 2019
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, LeakyReLU
from keras.layers import Activation, Dropout, Flatten, Dense,GlobalAveragePooling2D
from keras.optimizers import RMSprop
from keras.callbacks import *
from keras.models import Model
from keras import backend as K
from pathlib import Path
import os

###############################################################################
# HELPER FUNCTIONS
###############################################################################
def number_of_files(dirname):
	cpt = sum([len(files) for r, d, files in os.walk(dirname)])
	return cpt

################################################################################ 
# DÉFINITION DES DONNÉES D'ENTRÉE                                              #
################################################################################
train_data_dir = 'EuroSatRGB_training'
validation_data_dir = 'EuroSatRGB_validation'
test_data_dir = 'EuroSatRGB_test'
nb_train_samples=number_of_files(train_data_dir)
nb_validation_samples=number_of_files(validation_data_dir)
nb_test_samples=number_of_files(test_data_dir)
# Training image dimensions
img_width, img_height = 64, 64

epochs_pre = 10      # Pre-training epoch 
epochs_last = 20     # Complete model epoch
batch_size = 64 


################################################################################ 
# MODEL DEFINITION                                                             #
################################################################################
base_model = ResNet50(weights='imagenet', include_top=False) #Load the ResNet model

x = base_model.output
x = GlobalAveragePooling2D()(x)
# Add a fully connected layer
x = Dense(1024, activation='relu')(x)
x = Dropout(0.25)(x)
# and a logistic layer with 10 classes (in our dataset)
predictions = Dense(10, activation='softmax')(x)

# first: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = False

# Model definitions
model = Model(inputs=base_model.input, outputs=predictions)
model.summary()

# Compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=RMSprop(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

################################################################################ 
# IMAGES LOADING                                                               #
################################################################################

train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator() 

# Note, we could use data augmentation, 
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
	shuffle = True,
    class_mode='categorical') # Note: the class_mode is categorical

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
	shuffle = True,
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                             patience=5, min_lr=0.001)

tensor=TensorBoard(log_dir='.',histogram_freq=1,embeddings_freq=1,)
csv_logger = CSVLogger('resnet50_rgb_pre_log.csv', append=True, separator=';')

################################################################################ 
# RUN MODEL  (Part 1)                                                          #
################################################################################

# Start the pretraining 
original_hist=model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs_pre,
    verbose=1,
    callbacks=[csv_logger],
    validation_data=validation_generator,
    validation_steps= (nb_validation_samples // batch_size))

model.save('resnet50_rgb_first.hdf5')
# At this point, the top layers are well trained and we can start fine-tuning
# convolutional layers. We will freeze the bottom N layers
# and train the remaining top layers.

################################################################################ 
# RUN MODEL (Part 2)                                                           #
################################################################################

for layer in model.layers:
   layer.trainable = True

model.compile(optimizer=RMSprop(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

csv_logger = CSVLogger('resnet50_rgb_last_log.csv', append=True, separator=';')
checkpointer = ModelCheckpoint(filepath='resnet50_rgb_weights.{epoch:02d}-{val_acc:.2f}.hdf5', verbose=1, save_best_only=True)
original_hist2=model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs_last,
    verbose=1,
    callbacks=[csv_logger,checkpointer],
    validation_data=validation_generator,
    validation_steps= (nb_validation_samples // batch_size))

model.save("resnet50_rgb_end.h5")

################################################################################ 
# SAVE MODEL                                                                   #
################################################################################
model.save("resnet50_rgb_final.hdf5")
#
# Note: To load model:
# from keras.models import load_model 
# model=load_model("final_model.hdf5")

################################################################################ 
# FINAL NOTES                                                                  #
################################################################################
#
# 1. This demonstrate how to use transfer learning to train. However, we only use 
#    a very small part of the dataset. Using the full dataset, we can achieve 
#    > 95% accuracy.
# 

Using TensorFlow backend.
W0812 14:13:51.761715 140528245643136 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0812 14:13:51.805815 140528245643136 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0812 14:13:51.832106 140528245643136 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W0812 14:13:51.885252 140528245643136 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session inste

94658560/94653016 [==============================] - 3s 0us/step


W0812 14:14:04.974944 140528245643136 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0812 14:14:05.103070 140528245643136 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

W0812 14:14:08.151109 140528245643136 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
21/21 [==============================] - 35s 2s/step - loss: 1.9081 - acc: 0.6473 - val_loss: 2.3474 - val_acc: 0.4961
Epoch 2/10
21/21 [==============================] - 31s 1s/step - loss: 0.3043 - acc: 0.9127 - val_loss: 2.1996 - val_acc: 0.5000
Epoch 3/10
21/21 [==============================] - 31s 1s/step - loss: 0.2785 - acc: 0.9232 - val_loss: 2.6895 - val_acc: 0.4873
Epoch 4/10
21/21 [==============================] - 31s 1s/step - loss: 0.1826 - acc: 0.9544 - val_loss: 2.4309 - val_acc: 0.5085
Epoch 5/10
21/21 [==============================] - 31s 1s/step - loss: 0.1838 - acc: 0.9456 - val_loss: 1.7098 - val_acc: 0.6102
Epoch 6/10
21/21 [==============================] - 32s 2s/step - loss: 0.0874 - acc: 0.9710 - val_loss: 5.4283 - val_acc: 0.3633
Epoch 7/10
21/21 [==============================] - 31s 1s/step - loss: 0.1023 - acc: 0.9702 - val_loss: 2.4258 - val_acc: 0.5805
Epoch 8/10
21/21 [==============================] - 31s 1s/step - loss: 0.1755 - acc: 0.95

In [0]:
################################################################################ 
# END                                                                          #
################################################################################
# At this step, you should se a file named: resnet50_rgb_end.h5
!ls
################################################################################ 
# BUPLOAD THE GENERATED MODEL TO GOOGLE.DRIVE                                  #
################################################################################
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')
# Copy the resnet50_rgb_final.hdf5 to your google drive
!cp resnet50_rgb_final.hdf5 /content/drive/'My Drive'/

drive			   resnet50_rgb_last_log.csv
EuroSatRGB_test		   resnet50_rgb_pre_log.csv
EuroSatRGB_training	   resnet50_rgb_weights.01-0.82.hdf5
EuroSatRGB_validation	   resnet50_rgb_weights.08-0.83.hdf5
EuroSatRGB_very_small.zip  resnet50_rgb_weights.12-0.82.hdf5
resnet50_rgb_end.h5	   resnet50_rgb_weights.19-0.89.hdf5
resnet50_rgb_final.hdf5    sample_data
resnet50_rgb_first.hdf5
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
